In [1]:
import argparse
import ast
import json
import re
import time
from collections import Counter

import numpy as np

import sglang as sgl
from sglang.test.test_utils import (
    add_common_sglang_args_and_parse,
    select_sglang_backend,
)
from sglang.utils import dump_state_text, read_jsonl

INVALID = -9999999

/data/shibo/anaconda3/envs/sgl-py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/data/shibo/anaconda3/envs/sgl-py310/lib/python3.10/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
args = argparse.Namespace(
    data_path="sglang/benchmark/tree_of_thought_v0/test.jsonl",
    num_questions=2,
    port=10086,
    parallel=4,
    backend='srt',
    host="http://127.0.0.1",
    result_file="results.txt"
)

In [3]:
import random

In [82]:

def get_answer_value(answer_str):
    answer_str = answer_str.replace(",", "")
    numbers = re.findall(r"\d+", answer_str)
    if len(numbers) < 1:
        return INVALID
    try:
        return ast.literal_eval(numbers[-1])
    except SyntaxError:
        return INVALID


def most_frequent_number(numbers):
    if not numbers:
        return None

    frequency = Counter(numbers)
    most_frequent = max(frequency, key=frequency.get)
    return most_frequent


# Use a low temp to make the results more deterministic and the comparison more fair.
temp = 0.001


def propose_plan(s, question, num_branches):
    s += sgl.user(
        """Please generate a high-level plan for solving the following question. As the first step, just say what method and idea you will use to solve the question. You can reorganize the information in the question. Do not do the actual calculation. Keep your response concise and within 80 words. Question: """
        + question
    )
    forks = s.fork(num_branches)
    forks += sgl.assistant(sgl.gen("plan", max_tokens=256, temperature=temp))
    return forks


def execute_plan(s, num_branches):
    s += sgl.user(
        """The plan looks good! Now, use real numbers and do the calculation. Please solve the question step-by-step according to the high-level plan. Give me the final answer. Make your response short."""
    )
    forks = s.fork(num_branches)
    forks += sgl.assistant(sgl.gen("answer", max_tokens=256, temperature=temp))
    return forks


def reflect_solution(s, num_branches):
    s += sgl.user(
        """Okay. Now, evaluate your own solution and give it a score on a scale of 1 to 5. Please do rigorous check of the correctness."""
    )
    forks = s.fork(num_branches)
    forks += sgl.assistant(sgl.gen("score", max_tokens=256, temperature=temp))
    return forks


def get_final_answer(s, num_branches):
    s += sgl.user(
        """Based on your reflection, do you change your mind? Now, give me the final answer after careful consideration."""
    )
    forks = s.fork(num_branches)
    forks += sgl.assistant(sgl.gen("final_answer", max_tokens=256, temperature=temp))
    return forks


@sgl.function
def tree_search(s, question, num_branches):
    
    tree_log = []
    parent = []
    
    plan_forks = propose_plan(s, question, num_branches)

    sol_states = []
    for plan in plan_forks:
        forks = execute_plan(plan, num_branches)
        sol_states.extend(forks)
    
    tree_log.append(sol_states)

    ref_states = []
    for sol in sol_states:
        forks = reflect_solution(sol, num_branches)
        ref_states.extend(forks)

    ref_states = [k for k in ref_states if random.random() > 0.2]

    tree_log.append(ref_states)

    solutions = []
    for sol in ref_states:
        forks = get_final_answer(sol, num_branches)
        solutions.append(forks)
        
    tree_log.append(solutions)
    
    solutions = [[s.text() for s in forks] for forks in solutions]

    return tree_log

In [83]:
ret_values = tree_search.run_batch(
    arguments,
    temperature=0,
    backend=backend,
    num_threads=args.parallel,
    progress_bar=True,
)

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:08<00:00,  4.35s/it]


In [87]:
ret_value = ret_values[1]

print(len(ret_value.ret_value[0]),
        len(ret_value.ret_value[1]),
        len(ret_value.ret_value[2]))

4 7 7


In [94]:
print(ret_value.ret_value[0][0].text())

<|start_header_id|>user<|end_header_id|>

Please generate a high-level plan for solving the following question. As the first step, just say what method and idea you will use to solve the question. You can reorganize the information in the question. Do not do the actual calculation. Keep your response concise and within 80 words. Question: A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

To solve this problem, I will use the concept of ratios and proportions. I will first identify the ratio of blue fiber to white fiber, which is 2:1. Then, I will calculate the total number of bolts by adding the number of bolts of blue fiber to the number of bolts of white fiber.<|eot_id|><|start_header_id|>user<|end_header_id|>

The plan looks good! Now, use real numbers and do the calculation. Please solve the question step-by-step according to the high-level plan. Give me the final answer.

In [77]:
len(ret_values[0].ret_value)

3

In [95]:

lines = [l for l in read_jsonl(args.data_path)]

# Construct prompts
num_branches = 2
questions = []
labels = []
for i in range(len(lines[: args.num_questions])):
    questions.append(lines[i]["question"])
    labels.append(get_answer_value(lines[i]["answer"]))
assert all(l != INVALID for l in labels)
arguments = [{"question": q, "num_branches": num_branches} for q in questions]

# Select backend
backend = select_sglang_backend(args)

# Run requests
tic = time.time()
states = tree_search.run_batch(
    arguments,
    temperature=0,
    backend=backend,
    num_threads=args.parallel,
    progress_bar=True,
)
latency = time.time() - tic

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:07<00:00,  3.71s/it]


In [ ]:
answers_text = []
for s in states:
    answers_text.append([x for xs in s.ret_value for x in xs])

preds = []
for i in range(len(states)):
    answers = [get_answer_value(v) for v in answers_text[i]]
    preds.append(most_frequent_number(answers))

# Compute accuracy
acc = np.mean(np.array(preds) == np.array(labels))
invalid = np.mean(np.array(preds) == INVALID)
print(f"Latency: {latency:.3f}")
print(f"Invalid: {invalid:.3f}")
print(f"Accuracy: {acc:.3f}")

# Write results
dump_state_text(f"tmp_output_{args.backend}.txt", answers_text)

with open(args.result_file, "a") as fout:
    value = {
        "task": "tree_of_thought_gsm8k",
        "backend": args.backend,
        "num_gpus": 1,
        "latency": round(latency, 3),
        "accuracy": round(acc, 3),
        "num_requests": args.num_questions,
        "other": {
            "num_questions": args.num_questions,
            "parallel": args.parallel,
        },
    }
    fout.write(json.dumps(value) + "\n")


In [56]:
len(states)

2

In [51]:
type(states[0])

sglang.lang.interpreter.ProgramState

In [52]:
for m in states[0].messages():
    print(m["role"], ":", m["content"])

user : Please generate a high-level plan for solving the following question. As the first step, just say what method and idea you will use to solve the question. You can reorganize the information in the question. Do not do the actual calculation. Keep your response concise and within 80 words. Question: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?


In [ ]:
import sglang

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
import torch

good_token = ' +'
bad_token = ' -'
step_tag = '   ки'

path = "HanningZhang/Llama3.1-Math-PRM"
tokenizer = AutoTokenizer.from_pretrained(path)
plus_tag_id = tokenizer.encode(' +')[-1]
minus_tag_id = tokenizer.encode(' -')[-1]

candidate_tokens = [plus_tag_id,minus_tag_id]
step_tag_id = tokenizer.encode(f" {step_tag}")[-1] # 12902
model = AutoModelForCausalLM.from_pretrained(path).eval()


Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.71s/it]


In [108]:
step_tag_id

116624

In [98]:
input_for_prm = f"{question} {output1}"
input_id = torch.tensor([tokenizer.encode(input_for_prm)])

In [102]:
for i in input_id[0]:
    print(i.item(), tokenizer.decode(i))

128000 <|begin_of_text|>
18820 Jan
295 et
753 ’s
78878  ducks
11203  lay
220  
845 16
19335  eggs
824  per
1938  day
13 .
3005  She
50777  eats
2380  three
369  for
17954  breakfast
1475  every
6693  morning
323  and
293  b
2094 akes
55404  muff
1354 ins
369  for
1077  her
4885  friends
1475  every
1938  day
449  with
3116  four
13 .
3005  She
31878  sells
279  the
27410  remainder
520  at
279  the
20957  farmers
6 '
3157  market
7446  daily
369  for
400  $
17 2
824  per
7878  fresh
37085  duck
19151  egg
13 .
2650  How
1790  much
304  in
11441  dollars
1587  does
1364  she
1304  make
1475  every
1938  day
520  at
279  the
20957  farmers
6 '
3157  market
30 ?
15166  Step
220  
16 1
25 :
54765  Janet
596 's
78878  ducks
11203  lay
220  
845 16
19335  eggs
824  per
1938  day
13 .
116624  ки
8468 Step
220  
17 2
25 :
3005  She
50777  eats
2380  three
369  for
17954  breakfast
1475  every
6693  morning
11 ,
779  so
1364  she
706  has
220  
845 16
482  -
220  
18 3
284  =
220  
1032 13
1933

In [111]:
question = """Janet\u2019s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"""
output1 = """Step 1: Janet's ducks lay 16 eggs per day. киStep 2: She eats three for breakfast every morning, so she has 16 - 3 = 13 eggs left. киStep 3: She bakes muffins for her friends every day with four eggs, so she has 13 - 4 = 9 eggs left. киStep 4: She sells the remainder at the farmers' market daily for $2 per fresh duck egg, so she makes 9 * $2 = $18 every day at the farmers' market. The answer is: 18 ки""" # 18 is right
output2 = """Step 1: Janet's ducks lay 16 eggs per day. киStep 2: She eats three for breakfast every morning, so she has 16 - 3 = 13 eggs left. киStep 3: She bakes muffins for her friends every day with four eggs, so she has 13 - 4 = 9 eggs left. киStep 4: She sells the remainder at the farmers' market daily for $2 per fresh duck egg, so she makes 9 * $2 = $17 every day at the farmers' market. The answer is: 17 ки""" # 17 is wrong

output3 = """Step 1: Janet's ducks lay 16 eggs per day. Step 2: She eats three for breakfast every morning, so she has 16 - 3 = 13 eggs left. Step 3: She bakes muffins for her friends every day with four eggs, so she has 13 - 4 = 9 eggs left. Step 4: She sells the remainder at the farmers' market daily for $2 per fresh duck egg, so she makes 9 * $2 = $18 every day at the farmers' market. The answer is: 18 ки""" # 17 is wrong

for output in [output3]:
    input_for_prm = f"{question} {output}"
    input_id = torch.tensor([tokenizer.encode(input_for_prm)])

    with torch.no_grad():
        logits = model(input_id).logits[:,:,candidate_tokens]
        scores = logits.softmax(dim=-1)[:,:,0] 
        step_scores = scores[input_id == step_tag_id]
        print(step_scores)


tensor([0.9182])


In [118]:
text = '''If $x^3$ is a positive factor of $10!,$ how many possible integer values of $x$ are there?  (Reminder: For a positive integer $n$, the expression $n!$ stands for the product of the integers from 1 up to (and including) $n$.) 

To find the possible values of $x$, we need to find the prime factorization of $10!$. ки

The prime factorization of $10!$ is $2^8 \cdot 3^4 \cdot 5^2 \cdot 7$. ки

Since $x^3$ is a positive factor of $10!$, we need to find the possible values of $x$ such that $x^3$ divides $10!$. ки

We can start by finding the possible values of $x$ such that $x^3$ divides $2^8$. ки

The possible values of $x$ such that $x^3$ divides $2^8$ are $1, 2, 4, 8$. ки

We can then find the possible values of $x$ such that $x^3$ divides $3^4$. ки

The possible values of $x$ such that $x^3$ divides $3^4$ are $1, 3$. ки

We can then find the possible values of $x$ such that $x^3$ divides $5^2$. ки

The possible values of $x$ such that $x^3$ divides $5^2$ are $1, 5$. ки

We can then find the possible values of $x$ such that $x^3$ divides $7$. ки

The possible values of $x$ such that $x^3$ divides $7$ are $1$. ки

Now, we need to find the intersection of the possible values of $x$ for each prime factor. ки

The intersection of the possible values of $x$ for each prime factor is $1$. ки

Therefore, the possible values of $x$ are $1, 2, 4, 8, 3, 5$. ки

There are 6 possible values of $x$. ки

Answer: 6 ки'''

In [119]:
model.to("cuda:7")

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128001)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)


In [120]:
input_for_prm = f"{text}"
input_id = torch.tensor([tokenizer.encode(input_for_prm)]).cuda(7)

with torch.no_grad():
    logits = model(input_id).logits[:,:,candidate_tokens]
    scores = logits.softmax(dim=-1)[:,:,0] 
    step_scores = scores[input_id == step_tag_id]
    print(step_scores)

tensor([0.5330, 0.6286, 0.4628, 0.3091, 0.1489, 0.1437, 0.2133, 0.2332, 0.2703,
        0.2927, 0.3424, 0.2407, 0.2194, 0.1127, 0.2649, 0.3387],
       device='cuda:7')


In [ ]:
text = '''If $x^3$ is a positive factor of $10!,$ how many possible integer values of $x$ are there?  (Reminder: For a positive integer $n$, the expression $n!$ stands for the product of the integers from 1 up to (and including) $n$.) Step 1: To find the possible values of $x$, we need to find the prime factorization of $10!$. киStep 2: The prime factorization of $10!$ is $2^8 \cdot 3^4 \cdot 5^2 \cdot 7$. киStep 3: Since $x^3$ is a positive factor of $10!$, we need to find the possible values of $x$ such that $x^3$ divides $10!$. киStep 4: We can start by finding the possible values of $x$ such that $x^3$ divides $2^8$. киStep 5: The possible values of $x$ such that $x^3$ divides $2^8$ are $1, 2, 4, 8$. киStep 6: We can then find the possible values of $x$ such that $x^3$ divides $3^4$. киStep 7: The possible values of $x$ such that $x^3$ divides $3^4$ are $1, 3$. киStep 8: We can then find the possible values of $x$ such that $x^3$ divides $5^2$. киStep 9: The possible values of $x$ such that $x^3$ divides $5^2$ are $1, 5$. киStep 10: We can then find the possible values of $x$ such that $x^3$ divides $7$. киStep 11: The possible values of $x$ such that $x^3$ divides $7$ are $1$. киStep 12: Now, we need to find the intersection of the possible values of $x$ for each prime factor. киStep 13: The intersection of the possible values of $x$ for each prime factor is $1$. киStep 14: Therefore, the possible values of $x$ are $1, 2, 4, 8, 3, 5$. киStep 15: There are 6 possible values of $x$. The answer is: 6 ки'''

input_for_prm = text
input_id = torch.tensor([tokenizer.encode(input_for_prm)]).cuda(7)

with torch.no_grad():
    logits = model(input_id).logits[:,:,candidate_tokens]
    scores = logits.softmax(dim=-1)[:,:,0] 
    step_scores = scores[input_id == step_tag_id]
    print(step_scores)

tensor([0.5460, 0.7227, 0.7268, 0.6077, 0.4547, 0.4675, 0.5724, 0.5505, 0.5266,
        0.5241, 0.5907, 0.5705, 0.6346, 0.3756, 0.3568], device='cuda:7')


In [4]:
from transformers import AutoTokenizer

PROMPT = (
    "What is the range of the numeric output of a sigmoid node in a neural network?"
)

RESPONSE1 = "The output of a sigmoid node is bounded between -1 and 1."
RESPONSE2 = "The output of a sigmoid node is bounded between 0 and 1."

CONVS = [
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE1}],
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE2}],
]

tokenizer = AutoTokenizer.from_pretrained("Skywork/Skywork-Reward-Llama-3.1-8B-v0.2")
prompts = tokenizer.apply_chat_template(CONVS, tokenize=False)


In [5]:
prompts

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat is the range of the numeric output of a sigmoid node in a neural network?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThe output of a sigmoid node is bounded between -1 and 1.<|eot_id|>',
 '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat is the range of the numeric output of a sigmoid node in a neural network?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThe output of a sigmoid node is bounded between 0 and 1.<|eot_id|>']

In [16]:
import requests

url = "http://localhost:10087/judge"
data = {"model": "Skywork/Skywork-Reward-Llama-3.1-8B-v0.2", "text": prompts}

responses = requests.post(url, json=data).json()
for response in responses:
    # print(type(response))
    print(f"reward: {response['embedding'][0]}")

reward: -24.25
reward: 1.0703125


In [15]:
responses

[{'embedding': [-24.25],
  'meta_info': {'prompt_tokens': 69,
   'id': 'b40f91f07d0d41d2ab8a04fd5b2079a5'}},
 {'embedding': [1.109375],
  'meta_info': {'prompt_tokens': 69,
   'id': '57386a90ab164526b31fa958e9821de6'}}]

In [1]:
import torch

In [2]:
# !pip install accelerate

In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer


# Load model and tokenizer
device = "cuda:1"
model_name = "Skywork/Skywork-Reward-Llama-3.1-8B-v0.2"
rm = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map=device,
    # attn_implementation="flash_attention_2",
    num_labels=1,
)
rm_tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "Jane has 12 apples. She gives 4 apples to her friend Mark, then buys 1 more apple, and finally splits all her apples equally among herself and her 2 siblings. How many apples does each person get?"
response1 = "1. Jane starts with 12 apples and gives 4 to Mark. 12 - 4 = 8. Jane now has 8 apples.\n2. Jane buys 1 more apple. 8 + 1 = 9. Jane now has 9 apples.\n3. Jane splits the 9 apples equally among herself and her 2 siblings (3 people in total). 9 ÷ 3 = 3 apples each. Each person gets 3 apples."
response2 = "1. Jane starts with 12 apples and gives 4 to Mark. 12 - 4 = 8. Jane now has 8 apples.\n2. Jane buys 1 more apple. 8 + 1 = 9. Jane now has 9 apples.\n3. Jane splits the 9 apples equally among her 2 siblings (2 people in total). 9 ÷ 2 = 4.5 apples each. Each person gets 4 apples."

conv1 = [{"role": "user", "content": prompt}, {"role": "assistant", "content": response1}]
conv2 = [{"role": "user", "content": prompt}, {"role": "assistant", "content": response2}]

# Format and tokenize the conversations
# If you use `tokenize=False` with `apply_chat_template` and `tokenizer()` to tokenize the conversation,
# remeber to remove the duplicated BOS token.
conv1_tokenized = rm_tokenizer.apply_chat_template(conv1, tokenize=True, return_tensors="pt").to(device)
conv2_tokenized = rm_tokenizer.apply_chat_template(conv2, tokenize=True, return_tensors="pt").to(device)

# Get the reward scores
with torch.no_grad():
    score1 = rm(conv1_tokenized).logits[0][0].item()
    score2 = rm(conv2_tokenized).logits[0][0].item()
print(f"Score for response 1: {score1}")
print(f"Score for response 2: {score2}")

/data/shibo/anaconda3/envs/sgl-py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/data/shibo/anaconda3/envs/sgl-py310/lib/python3.10/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.12s/it]


torch.Size([1, 179, 4096])
torch.Size([1, 179, 1])
torch.Size([1, 1])
torch.Size([1, 179, 4096])
torch.Size([1, 179, 1])
torch.Size([1, 1])
Score for response 1: 13.8125
Score for response 2: -9.25


In [ ]:
# torch.Size([1, 179, 4096])
# torch.Size([1, 179, 1])
# torch.Size([1, 1])
# torch.Size([1, 179, 4096])
# torch.Size([1, 179, 1])
# torch.Size([1, 1])

In [5]:
output = rm(conv1_tokenized)

In [7]:
from transformers import LlamaForSequenceClassification